## Mod 2 Assessment

In [1]:
# put all imports here
import json
import requests
import pandas as pd
import pymongo
import sqlite3
from bs4 import BeautifulSoup
from bson.json_util import loads

### SQL (10 Minutes)
There is a sqlite3 database in `assets/books.db`. The SQL to create this is also in `assets/books.sql`, if you want to run it manually (you can also import this using https://sqliteonline.com/, or run the SQL file directly).  Both have the same schema and data.

The schema has three tables. You can explore the schema in the fiddle posted above.  Please answer the following questions.

1. Connect to the database using sqlite3

In [2]:
# Your code here
conn = sqlite3.connect('assets/books.db')
c =conn.cursor( )


2. Querying the DB: how many pages are in the book "Nine Stories"?

In [3]:
# Your code here
sql_1 = """
SELECT * FROM sqlite_master WHERE type="table"
"""
c.execute(sql_1)
df_master = pd.DataFrame(c.fetchall())
df_master.columns = [x[0] for x in c.description]
df_master


,type,name,tbl_name,rootpage,sql
0,table,author,author,2,"CREATE TABLE author (\n author_id INT,\n ..."
1,table,book,book,3,"CREATE TABLE book (\n book_id INT,\n tit..."
2,table,book_author,book_author,4,"CREATE TABLE book_author (\n author_id INT,..."


In [4]:
field_dict = {}

for tab_name in df_master.name:
    sql_2 = """

    SELECT * FROM {}

    """.format(tab_name)
    c.execute(sql_2)
    field_dict.update({tab_name: [x[0] for x in c.description]})

    
print("table names: {}".format(list(field_dict.keys())))
print("\n")

for tab_name in field_dict.keys():
    print("""columns in table "{}": {}\n""".format(tab_name, field_dict[tab_name]))

table names: ['author', 'book', 'book_author']


columns in table "author": ['author_id', 'name', 'country']

columns in table "book": ['book_id', 'title', 'pages']

columns in table "book_author": ['author_id', 'book_id']



In [ ]:
sql_3 = """
SELECT * FROM book COUNT(pages) as num_pages
WHERE title ='Nine Stories'
"""
c.execute(sql_3)
# df_master1 = pd.DataFrame(c.fetchall())
# df_master1.columns = [x[0] for x in c.description]
# df_master1

3. How many authors are from the USA?

In [9]:
# Your code here
c.execute("""SELECT  count(DISTINCT(name)) AS distinct_author, count(name) as total_authors FROM author WHERE country = 'USA'; """)
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df





,distinct_author,total_authors
0,6,6


In [ ]:
There are  six authors from the USA

4. How many authors does the book "Professional ASP.NET 4.5 in C# and VB" have?

In [15]:
# Your code here
c.execute("""SELECT book_id count(book_id) AS distinct_author FROM book WHERE title = "Professional ASP.NET 4.5 in C# and VB"; """)
df4 = pd.DataFrame(c.fetchall())
df4.columns = [i[0] for i in c.description]
df4

OperationalError: near "(": syntax error

5. How many pages total have been written by non-American authors?

In [ ]:
# Your code here


### Object Oriented Programming (15 Minutes)
Below is the stub of some classes. Make the classes adhere to the following rules. The code beneath the cell should  run successfully.
1. Force every new instance of `WeWorkMember` to pass in a value to set the attribute `name`.
1. Give every new instance of `WeWorkMember` an attribute `caffeinated` that is set to `False`. 
1. Give `WeWorkMember` an **instance** method called `caffeinate()` that prints out "Getting coffee!" and sets the attribute `caffeinated` to `True`.
1. Make it so that `Staff` and `Student` inherit from `WeWorkMember`
1. Give `Staff` a **static** method called `cheer()` that prints out "Goooooooo Flatiron Students!"
1. Give `Students` a **class** method called `learn()` that takes in an integer and returns that number +1

In [ ]:
# Modify this cell
class WeWorkMember:
    caffeinated = False

class Staff:
    caffeinated = False

class Student:
    caffeinated = False
    def caffeinate(self, Staff, Student):
        print("Getting coffee!")
        self.caffeinate = True
        self.Staff = Staff
        self.Student = Student
        
        

In [ ]:
# DO NOT MODIFY this cell. if you want to add more tests, add them in a new cell!
try:
    nobody = Staff()
    raise RuntimeError('Staff needs a name!')
except TypeError:
    pass

andy = Staff('Andy')
assert andy.caffeinated is False
andy.caffeinate()
assert andy.caffeinated is True

Staff.cheer()

bobby = Student('Bobby Tables')
assert bobby.caffeinated is False
bobby.caffeinate()
assert bobby.caffeinated is True

Student.learn(99)

### APIs (10 Minutes)
Using this API (http://www.nokeynoshade.party/api/judges) about RuPaul's Drag Race (docs: https://drag-race-api.readme.io/docs/get-all-judges), tell me how many judges of each **`type`** there were in the first 50 records the API returns.
 - the 50 judges will have **`id`** ranging from 1 to 53
 - you can do the aggregation in pure Python, Pandas, SQL-- whatever's easiest for you

In [18]:

api_key = 'http://www.nokeynoshade.party/api/judges'
url = 'https://drag-race-api.readme.io/docs/get-all-judges'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


response = requests.get(url, headers=headers)
print(response)
print(type(response.text))
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
<class 'str'>
<!DOCTYPE html><html ng-app="hub" lang="en"><head><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="readme-deploy" content="e9ebbd4d2482daf0d43abf4c4ae3658d13172095"><meta name="readme-subdomain" content="drag-race-api"><meta name="readme-version" content="1.0"><title ng-bind="pageTitle">Get All Judges</title><meta name="description" content="RuPaul's Drag Race API, containing information about the Queens, Episodes, Seasons, Challenges, Lipsyncs, and Judges of the TV series."><meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no"><meta property="og:title" content="Get All Judges"><meta property="og:description" content="Get a list of judges in the entire herstory of RuPaul's Drag Race.  |Params |Example |Default| |-|-:|-:| |Limit |50| 25| |Offset| 25| 0|  _example request:_ GET `http://www.nokeynoshade.party/api/judges`  _example response:_ ```json [    

# Web Scraping (15 Minutes)
Scrape the website [http://quotes.toscrape.com/](http://quotes.toscrape.com/).  Turn each quote on the page into a Pandas dataframe consisting of two columns: the author of the quote and the quote itself. The final result should be a dataframe of 10 rows (not counting header) and 2 columns (not counting index).

In [ ]:
forcast_date = []
for date in game_dates[0]:
    url = 'http://quotes.toscrape.com/'
    resp = requests.get(url)
   

### NoSQL (10 Minutes)
Answer all of the following questions by querying Mongo and manipulating the results in Python
1. Load data from `assets/grades.json` into Mongo (this code is written for you)

In [ ]:
# you shouldn't need to edit this cell!
with open('assets/grades.jsonl') as f:
    # loads() comes from the bson library
    file_data = [loads(line) for line in f.readlines()]

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mod2db"]
coll = db["testcoll"]

coll.insert_many(file_data)

2. How many records are there total?

In [ ]:
# Your code here


3. How many students have taken the class with `class_id` = **29**?

In [ ]:
# Your code here


4. Super bonus question: for student **12** in class **23**, what grade did they get on their exam?

In [ ]:
# Your code here


In [ ]:
# for when you're done with this portion, this deletes the data we added.
client.drop_database("mod2db")

## Assessment submission (2 Minutes)
Please save your completed file as `mod2-assessment.ipynb` and upload it using [this form](https://docs.google.com/forms/d/e/1FAIpQLSczNVM9Q6Dr3w1YR3kkg4PNaOsfDO-qIvh1wLy6rR0SEwTwIQ/viewform?usp=sf_link)